# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [41]:
from datasets import load_dataset

# Nombre del dataset en el hub de Hugging Face
dataset_name = "mrm8488/CHISTES_spanish_jokes"

# Cargar el dataset
dataset = load_dataset(dataset_name, split='train').train_test_split(test_size = 0.2)

# Mostrar un ejemplo del dataset
print(dataset['train'][0])

{'id': 1149, 'text': 'La esposa que llama desesperada y molesta a la recepción del hotel: \n- Por favor vengan rápido que estoy teniendo una discusión con mi esposo y él dice que va a saltar por la ventana.\nDe la recepción le responden:\n- Señora, ese es un asunto personal.\nY la esposa contesta:\n- Sí, señor, pero la ventana no se abre, y eso ya es un problema de mantenimiento de ustedes.', 'keywords': 'hoteles,suicidios', 'funny': 3, 'category': 'familia'}


In [42]:
print(dataset['train'][0:10])  
print(dataset)  


{'id': [1149, 2058, 1615, 2278, 890, 2109, 1, 1313, 2141, 208], 'text': ['La esposa que llama desesperada y molesta a la recepción del hotel: \n- Por favor vengan rápido que estoy teniendo una discusión con mi esposo y él dice que va a saltar por la ventana.\nDe la recepción le responden:\n- Señora, ese es un asunto personal.\nY la esposa contesta:\n- Sí, señor, pero la ventana no se abre, y eso ya es un problema de mantenimiento de ustedes.', '- Padre, anoche mi primo me hizo suya.\n- Contra tu voluntad?\n- No.... contra la pared.', 'No sé por qué cada vez que tengo sexo con mis novias gritan el mismo nombre de otra persona. ¿Alguien sabe quién es "Socorro Meviolan"?', '- Doctor, creo que tengo complejo de fea. \n- De complejo nada.', 'Un loco entra a una panadería y le pregunta al panadero:\n- Señor, ¿ya salió el pan?\nY el panadero le dice:\n- Sí, ya salió.\nEntonces el loco le pregunta:\n- ¿Y a qué hora regresará?', 'En la clase de gramática la maestra le pregunta a Jaimito.\n-Jaim

In [43]:
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorWithPadding


def process_lines(text_input):
    # Tokeniza el texto
    tokenized_text = tokenizer(
        text_input["text"],
        padding="max_length",
        max_length=1024,  # Establece la longitud máxima que tu modelo puede manejar
        truncation=True
    )  
    # Añade la etiqueta 'funny' a las salidas del tokenizer bajo la clave correcta 'labels'
    tokenized_text['labels'] = text_input['funny']-1  # adjust labes to index number starting from 0
    return tokenized_text



##model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

splits=["train","test"]

'''
#Elimina la columna 'id' del dataset 
dataset["train"] = dataset["train"].remove_columns("id")
dataset["train"] = dataset["train"].remove_columns("keywords")
dataset["train"] = dataset["train"].remove_columns("category")

dataset["test"] = dataset["test"].remove_columns("id")
dataset["test"] = dataset["test"].remove_columns("keywords")
dataset["test"] = dataset["test"].remove_columns("category")
'''

tokenized_dataset = {}
for split in dataset.keys():
    # Aplica la función de tokenización y etiquetado a cada split del dataset
    tokenized_dataset[split] = dataset[split].map(process_lines)
    tokenized_dataset[split] = tokenized_dataset[split].remove_columns("funny")

print(tokenized_dataset['train'][0])



Map:   0%|          | 0/1935 [00:00<?, ? examples/s]

Map:   0%|          | 0/484 [00:00<?, ? examples/s]

{'id': 1149, 'text': 'La esposa que llama desesperada y molesta a la recepción del hotel: \n- Por favor vengan rápido que estoy teniendo una discusión con mi esposo y él dice que va a saltar por la ventana.\nDe la recepción le responden:\n- Señora, ese es un asunto personal.\nY la esposa contesta:\n- Sí, señor, pero la ventana no se abre, y eso ya es un problema de mantenimiento de ustedes.', 'keywords': 'hoteles,suicidios', 'category': 'familia', 'input_ids': [14772, 1658, 1930, 64, 8358, 32660, 1689, 748, 274, 525, 4763, 331, 18605, 18059, 257, 8591, 1407, 79, 979, 18840, 1619, 7541, 25, 220, 198, 12, 20139, 2661, 8710, 1030, 374, 6557, 79, 17305, 8358, 1556, 726, 3478, 72, 31110, 555, 64, 1221, 385, 72, 18840, 369, 21504, 1658, 1930, 78, 331, 38251, 75, 17963, 8358, 46935, 257, 8268, 283, 16964, 8591, 7435, 2271, 13, 198, 5005, 8591, 1407, 79, 979, 18840, 443, 3031, 268, 25, 198, 12, 1001, 12654, 5799, 11, 1658, 68, 1658, 555, 355, 403, 1462, 2614, 13, 198, 56, 8591, 1658, 1930, 64,

In [44]:
print("Tamaño del dataset de entrenamiento:", len(dataset["train"]))
print("Tamaño del dataset de prueba:", len(dataset["test"]))

Tamaño del dataset de entrenamiento: 1935
Tamaño del dataset de prueba: 484


In [45]:
print(dataset["train"][0])  # Imprime el primer ejemplo para ver la estructura
print(dataset["test"][0])   # Imprime el primer ejemplo para ver la estructura

{'id': 1149, 'text': 'La esposa que llama desesperada y molesta a la recepción del hotel: \n- Por favor vengan rápido que estoy teniendo una discusión con mi esposo y él dice que va a saltar por la ventana.\nDe la recepción le responden:\n- Señora, ese es un asunto personal.\nY la esposa contesta:\n- Sí, señor, pero la ventana no se abre, y eso ya es un problema de mantenimiento de ustedes.', 'keywords': 'hoteles,suicidios', 'funny': 3, 'category': 'familia'}
{'id': 395, 'text': '- Cariño, ya acosté al niño y está la cena en el horno\n- Pero si en la cama hay un pollo...\n- Apaga el horno!!! APAGA EL HORNO!!!', 'keywords': 'hornos,niños,pollos', 'funny': 1, 'category': 'familia'}


In [46]:
print(tokenized_dataset["train"].features) 
# checking features

print(tokenized_dataset['train'][0])

print(tokenized_dataset['train'][0]['input_ids'])
print(tokenized_dataset['train'][0]['labels'])

{'id': Value(dtype='int64', id=None), 'text': Value(dtype='string', id=None), 'keywords': Value(dtype='string', id=None), 'category': Value(dtype='string', id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'labels': Value(dtype='int64', id=None)}
{'id': 1149, 'text': 'La esposa que llama desesperada y molesta a la recepción del hotel: \n- Por favor vengan rápido que estoy teniendo una discusión con mi esposo y él dice que va a saltar por la ventana.\nDe la recepción le responden:\n- Señora, ese es un asunto personal.\nY la esposa contesta:\n- Sí, señor, pero la ventana no se abre, y eso ya es un problema de mantenimiento de ustedes.', 'keywords': 'hoteles,suicidios', 'category': 'familia', 'input_ids': [14772, 1658, 1930, 64, 8358, 32660, 1689, 748, 274, 525, 4763, 331, 18605, 18059, 257, 8591, 1407, 79, 979, 18840, 1619, 7541, 25, 220, 198, 12, 20139, 2661, 8

In [47]:
# Extrae todos los valores de 'labels'
labels = tokenized_dataset['train'][:]["labels"]

# Encuentra los valores únicos en la columna 'labels'
unique_labels = set(labels)
print("Valores únicos en 'labels':", unique_labels)

Valores únicos en 'labels': {0, 1, 2}


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [48]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained( "gpt2", num_labels=3)
model.config.pad_token_id = model.config.eos_token_id # add config line to avoid future error

# These was wrong, for this model 
# from transformers import GPT2LMHeadModel
# model = GPT2LMHeadModel.from_pretrained('gpt2')
# model = AutoModelForCausalLM.from_pretrained( "gpt2")
    
print (model)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=3, bias=False)
)


In [49]:
# Configuring Lora
import peft
from peft import LoraConfig, get_peft_model, PeftModel

config = LoraConfig(
        r=32,
        lora_alpha=32,
        lora_dropout=0.05,
        bias="none",
        task_type="SEQ_CLS",
        fan_in_fan_out = True,
        # tune the embedding layer and prediction head
        target_modules=["c_attn", "c_proj"]
    )
#Create the PEFT model.

peft_model = get_peft_model(model, config)

#Conglar los parámetros
for param in peft_model.parameters():
    param.requires_grad = False
    
# Habilitar el entrenamiento solo para los parámetros del lm_head
#for param in peft_model.lm_head.parameters():
#    param.requires_grad = True    


In [50]:
print(peft_model.print_trainable_parameters())

trainable params: 0 || all params: 127,688,448 || trainable%: 0.0
None


In [51]:
print (peft_model)

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): GPT2ForSequenceClassification(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): Linear(
                in_features=768, out_features=2304, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embeddi

In [52]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = (preds == labels).mean()
    return {"accuracy": accuracy}

In [53]:
#cheching
print(tokenized_dataset['train'][10]['input_ids'])   
print(tokenized_dataset['train'][10]['labels'])   

[12, 9356, 11, 1587, 123, 421, 2634, 443, 267, 22019, 260, 257, 21504, 1667, 17305, 30, 220, 198, 10, 1778, 1667, 17305, 279, 671, 344, 30972, 11815, 1332, 13174, 13, 220, 198, 12, 1587, 123, 46, 83, 11815, 1332, 13174, 30, 220, 198, 10, 311, 8836, 11, 267, 5948, 2376, 8358, 443, 5466, 390, 22346, 37409, 85, 418, 13, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50

In [54]:
print(tokenized_dataset["train"].features) 
# checking features


{'id': Value(dtype='int64', id=None), 'text': Value(dtype='string', id=None), 'keywords': Value(dtype='string', id=None), 'category': Value(dtype='string', id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'labels': Value(dtype='int64', id=None)}


In [55]:
from transformers import TrainingArguments, Trainer, TrainerCallback, pipeline, DataCollatorWithPadding


training_args = TrainingArguments(
    output_dir="./data/spam_not_spam",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,    
    num_train_epochs=1,
#    fp16=True,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    weight_decay = 0.01,
 #   save_steps=10_000,
 #   save_total_limit=2,
 #   gradient_checkpointing=True,
    load_best_model_at_end=True
    )


trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer) 
)

eval_results = trainer.evaluate()
print(f"Initial Evaluation Results: {eval_results}")





You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Initial Evaluation Results: {'eval_loss': 1.646266222000122, 'eval_runtime': 45.2391, 'eval_samples_per_second': 10.699, 'eval_steps_per_second': 0.685}


## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [60]:
from datasets import Dataset

text= dataset["test"][0]["text"]
tokenized_text= tokenized_dataset['test'][0]
#tokenized_text = Dataset.from_dict( tokenized_dataset['test'][0])
print(text)
print(tokeniez_text)



- Cariño, ya acosté al niño y está la cena en el horno
- Pero si en la cama hay un pollo...
- Apaga el horno!!! APAGA EL HORNO!!!
[22362, 78, 1658, 449, 1385, 10094, 551, 1288, 763, 1455, 952, 3671, 794, 25440, 551, 537, 589, 2376, 8358, 875, 544, 8591, 1534, 274, 5799, 25, 198, 12, 34153, 12654, 418, 11, 31215, 17266, 12654, 2271, 256, 1734, 271, 8358, 8710, 343, 19750, 312, 418, 390, 1216, 315, 292, 267, 3326, 67, 17786, 31215, 18466, 9226, 11, 1587, 123, 2934, 936, 15573, 4598, 30, 198, 2348, 288, 544, 43237, 84, 1153, 68, 11, 8591, 1534, 274, 5799, 1556, 6557, 5720, 25440, 401, 78, 5719, 19750, 312, 418, 2341, 435, 4182, 418, 25, 198, 12, 126, 94, 39, 5708, 16852, 28265, 1587, 123, 2934, 8358, 34439, 19750, 17305, 30, 198, 12, 5005, 284, 9830, 384, 12654, 273, 5350, 198, 12, 126, 123, 56, 256, 21356, 11, 17409, 30, 198, 12, 126, 94, 38101, 23291, 390, 443, 354, 30302, 0, 198, 25189, 533, 344, 449, 1385, 10094, 748, 77, 12003, 1224, 83, 3263, 68, 331, 369, 555, 64, 781, 273, 551, 12

In [61]:

prediccion = trainer.predict(test_dataset=tokenized_text)

KeyError: 0

In [33]:
import torch

with torch.no_grad():  # Desactivar el cálculo de gradientes para inferencia
    outputs = trainer(**tokenized_text)
    logits = outputs.logits  # La salida son los logits

# Convertir los logits a probabilidades usando softmax
probabilities = torch.softmax(logits, dim=-1)
predicted_label = torch.argmax(probabilities, dim=-1)

print("Predicted label:", predicted_label.item())

TypeError: 'Trainer' object is not callable